In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
objp = np.zeros((6*8,3), np.float32)
# print(objp)
objp[:,:2] = np.mgrid[0:8, 0:6].T.reshape(-1,2)

objpoints = [] # 실세계 3차원 지점(객체지점)
imgpoints = [] # 이미지평면 2차원 이미지 지점

In [3]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

images = glob.glob('calibration_wide/GO*.jpg')
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)
    
    if ret == True:
        objpoints.append(objp)
        
        corners2 = cv2.cornerSubPix(gray, corners,(5,5),(-1,-1),criteria)
        imgpoints.append(corners2)#이미지 지점 입력
        cv2.drawChessboardCorners(img, (8,6), corners2, ret)
cv2.destroyAllWindows()

In [4]:
t, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,
                                                   imgpoints,
                                                   gray.shape[::-1],
                                                 None,None)
print(mtx)

[[560.22556619   0.         651.15623038]
 [  0.         561.2815969  498.96859625]
 [  0.           0.           1.        ]]


In [5]:
import pickle
%matplotlib inline

In [60]:
cap = cv2.VideoCapture('redball.mp4')
# height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
# width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT,640)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH,480)
# print(height)
# print(width)
ret, src = cap.read()
print(src.shape[0])


# print('fx, fy:',mtx[0][0],mtx[1][1])

1080


In [7]:
#왜곡제거
dst = cv2.undistort(src,mtx,dist,None,mtx)
dst = cv2.resize(dst,dsize=(640,640))
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [8]:
tot_error=0
for i in range(len(objpoints)):
    imgpoints2,_=cv2.projectPoints(objpoints[i],rvecs[i],
                                  tvecs[i],mtx,dist)
    error=cv2.norm(imgpoints[i],imgpoints2,cv2.NORM_L2)/len(imgpoints2)
    tot_error += error
    
print("total error : ",tot_error/len(objpoints))

total error :  0.07172472745558958


In [12]:
from imutils import paths
import numpy as np
import imutils

In [38]:
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray,(5,5),0)
edged = cv2.Canny(gray, 70, 180)

cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key = cv2.contourArea)
marker = cv2.minAreaRect(c)

#bbox그리기
box = cv2.boxPoints(marker) if imutils.is_cv2() else cv2.boxPoints(marker)
box = np.int0(box)
#좌측상단부터 반시계방향 좌표
cor1, cor2, cor3, cor4 = box
centerX,centerY = ((cor1 + cor4)/2 + (cor2 + cor3)/2)/2
cx,cy = int(centerX),int(centerY)
print(centerX,centerY)
cv2.drawContours(src, [box], -1, (0,255,0), 2)
cv2.circle(src,(cx,cy), 1, (0,255,0), 2)


cv2.imshow('src',src)
cv2.waitKey()
cv2.destroyAllWindows()

605.75 497.25


In [39]:
import PIL.ExifTags
import PIL.Image

In [40]:
#초점거리 구하기
exif_img = PIL.Image.open(images[0])
exif_data = {
	PIL.ExifTags.TAGS[k]:v
	for k, v in exif_img._getexif().items()
	if k in PIL.ExifTags.TAGS}

In [64]:
fx = mtx[0][0]
fy = mtx[1][1]

# fx = 초점길이(f) x 배율인수(mx)
m = ((fx+fy)/2) / 3.0 # 단위 : px/mm
print('m:',m,'fx:',fx,'fy:',fy)

# 개체크기(width픽셀) 구해야 한다.
# 사진의 기본 해상도(체스판사진세부사항) : m = 
# 개체를촬영한비디오 해상도(카메라로촬영한 비디오) : x
# x 값이 저해상도에서 배율인수
x = 1080*m/1280
print('x :',x)

# minAreaRect를 통해서 개체크기(width 픽셀값)을 구한다
#따라서 개체크기/x = 이미지센서의 물체크기(mm)
objectWidth = np.sqrt((cor4[0]-cor1[0])**2 + (cor4[1]-cor1[1])**2)
imgSensor = objectWidth/x
print('objectWidth : ',objectWidth)
print('imgSensor : ',imgSensor)

m: 186.9178605147101 fx: 560.2255661908922 fy: 561.2815968973683
x : 157.71194480928665
objectWidth :  633.7586291325745
imgSensor :  4.0184567497341295


In [76]:
# 카메라에서 개체까지의 거리
dist = 380 * 3.0 / imgSensor
print('물체와 카메라 사이의 실제 거리 %dcm' %(round(dist/10,2)))


물체와 카메라 사이의 실제 거리 28cm
